In [1]:
import os
os.chdir("/home/sunflower124/25-26_internship/jaxlogit")

In [47]:
import pandas as pd
import jax
import numpy as np
import random

from jaxlogit.mixed_logit import MixedLogit, ConfigData
from sklearn.model_selection import train_test_split

In [15]:
import os
os.chdir("/home/evelyn/projects_shared/jaxlogit")

In [105]:
df = pd.read_csv("examples/electricity_short.csv")

In [117]:
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 5000

X = np.array(df[varnames])
y = df['choice']

panels = df['id']

In [112]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/electricity_long.csv")

In [113]:
df = pd.read_csv("https://raw.github.com/arteagac/xlogit/master/examples/data/electricity_long.csv")
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 600

X = np.array(df[varnames])
y = np.array(df['choice'])

choice_situations = X.shape[0]
number_chosen = int(((choice_situations * 0.8) // 4) * 4)
chosen = random.sample(range(1, choice_situations), number_chosen)
selected = np.array([1 if i in chosen else 0 for i in range(0, choice_situations)]).astype(bool)

X_train, X_test, y_train, y_test, ids_train, ids_test, alts_train, alts_test = train_test_split(X, y, np.array(df['chid']), np.array(df['alt']), test_size=0.201, random_state=1, stratify=None)

# X_train = X[selected, :]
# X_test = X[~selected, :]
# y_train = y[selected]
# y_test = y[~selected]

# ids_train = df['chid'][:number_chosen]
# ids_test = df['chid'][~selected]
# alts_train = df['alt'][selected]
# alts_test = df['alt'][~selected]

X_train = X[:number_chosen, :]
X_test = X[number_chosen:, :]
y_train = y[:number_chosen]
y_test = y[number_chosen:]

ids_train = df['chid'][:number_chosen]
ids_test = df['chid'][number_chosen:]
alts_train = df['alt'][:number_chosen]
alts_test = df['alt'][number_chosen:]

panels_train = df['id'][:number_chosen]

In [106]:
dfc = df.groupby('id')['choice'].sum()
dfd = dfc.index[dfc == 12]
df = df[df['id'].isin(dfd)]

varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
n_draws = 600

X = np.array(df[varnames])
y = df['choice']

In [56]:
print(X_train.shape)
print(X_train)
print(y_train)
print(ids_train)
print(alts_train)

(13784, 6)
[[7 5 0 1 0 0]
 [9 1 1 0 0 0]
 [0 0 0 0 0 1]
 ...
 [0 0 0 0 1 0]
 [0 5 1 0 0 1]
 [9 5 0 1 0 0]]
[0 0 0 ... 0 1 0]
0           1
1           1
2           1
3           1
4           2
         ... 
13779    3445
13780    3446
13781    3446
13782    3446
13783    3446
Name: chid, Length: 13784, dtype: int64
0        1
1        2
2        3
3        4
4        1
        ..
13779    4
13780    1
13781    2
13782    3
13783    4
Name: alt, Length: 13784, dtype: int64


In [107]:
model = MixedLogit()

config = ConfigData(
    panels=df['id'],
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)

model.fit(
    X=X,
    y=y,
    varnames=varnames,
    ids=df['chid'],
    alts=df['alt'],
    randvars={'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'},
    config=config
)
display(model.summary())

    Message: CONVERGENCE: RELATIVE REDUCTION OF F <= FACTR*EPSMCH
    Iterations: 42
    Function evaluations: 61
Estimation time= 1.0 seconds
---------------------------------------------------------------------------
Coefficient              Estimate      Std.Err.         z-val         P>|z|
---------------------------------------------------------------------------
pf                     -5.8668254     1.0000000    -5.8668256      7.64e-05 ***
cl                      1.2070932     1.0000000     1.2070931         0.251    
loc                   348.2359950     1.0000000   348.2359924      2.12e-25 ***
wk                    346.5074446     1.0000000   346.5074463      2.25e-25 ***
tod                   -37.2957049     1.0000000   -37.2957039      8.87e-14 ***
seas                  -34.3221384     1.0000000   -34.3221397      2.38e-13 ***
sd.pf                -150.0327768     1.0000000  -150.0327759      5.16e-21 ***
sd.cl                -149.9156955     1.0000000  -149.9156952      5.

None

In [ ]:
pf                     -0.9972102     0.0361699   -27.5701452      7.2e-153 ***
cl                     -0.2196812     0.0143577   -15.3006166      2.44e-50 ***
loc                     2.2901807     0.0891457    25.6903002     3.37e-134 ***
wk                      1.6943247     0.0719012    23.5646304     2.87e-114 ***
tod                    -9.6752279     0.3117174   -31.0384572     1.15e-189 ***
seas                   -9.6961836     0.3111496   -31.1624480     4.94e-191 ***
sd.pf                   0.2207255     0.0126686    17.4230731      1.54e-64 ***
sd.cl                   0.4115547     0.0200678    20.5082053       5.5e-88 ***
sd.loc                  1.7840255     0.0974403    18.3089162      6.13e-71 ***
sd.wk                   1.2296227     0.0838150    14.6706716      1.93e-46 ***
sd.tod                  2.2757059     0.1257114    18.1026193      2.01e-69 ***
sd.seas                 1.4862206     0.1316147    11.2922079      4.06e-28 ***

In [75]:
varnames = ['pf', 'cl', 'loc', 'wk', 'tod', 'seas']
varnames = ['pf', 'loc', 'wk', 'tod', 'seas']
X_restricted_train = X_train[:, [0,2,3,4,5]]

model = MixedLogit()

config = ConfigData(
    panels=None,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)

model.fit(
    X=X_restricted_train,
    y=y_train,
    varnames=varnames,
    ids=ids_train,
    alts=alts_train,
    randvars={'pf': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'},
    config=config
)
display(model.summary())

ValueError: inconsistent alts values in long format, expected [1 2 3 ... 2 3 4], got [1 2 3 4]

In [118]:
config = ConfigData(
    panels=panels,
    n_draws=n_draws,
    skip_std_errs=True,  # skip standard errors to speed up the example
    batch_size=None,
    optim_method="L-BFGS-B",
)
init_coeff =  np.array([-0.9972102,
-0.2196812,
 2.2901807,
 1.6943247,
-9.6752279,
-9.6961836,
 0.2207255,
 0.4115547,
 1.7840255,
 1.2296227,
 2.2757059,
 1.4862206,])
config.init_coeff = init_coeff

In [119]:
prob = model.predict(X, varnames, df['alt'], df['chid'], {'pf': 'n', 'cl': 'n', 'loc': 'n', 'wk': 'n', 'tod': 'n', 'seas': 'n'}, config)

In [121]:
for i in range(prob.shape[0]):
    print(f"{prob[i]} : {y[i]}")

[0.2985409  0.33100215 0.17472473 0.19573218] : [0 0 0 1]
[0.4274626  0.19668742 0.2790752  0.09677478] : [0 0 1 0]
[0.13355643 0.4129085  0.1913335  0.26220155] : [0 0 0 1]
[0.17287637 0.05846132 0.5577994  0.21086285] : [0 0 0 1]
[0.32511258 0.22880483 0.09524423 0.3508384 ] : [1 0 0 0]
[0.11442507 0.17876574 0.30471605 0.40209314] : [0 0 0 1]
[0.19205786 0.20167194 0.16377215 0.442498  ] : [1 0 0 0]
[0.35739267 0.16550004 0.32969415 0.14741318] : [0 0 1 0]
[0.26995263 0.12951706 0.38784948 0.21268082] : [1 0 0 0]
[0.20774734 0.28650934 0.1941893  0.311554  ] : [0 1 0 0]
[0.24732263 0.2774156  0.23530185 0.23995996] : [0 0 1 0]
[0.22780201 0.14506558 0.34725907 0.27987334] : [0 0 0 1]
[0.32589877 0.35013688 0.09076969 0.23319468] : [0 0 0 1]
[0.14663704 0.22752573 0.27922577 0.34661147] : [0 0 0 1]
[0.14229897 0.09487493 0.41782558 0.34500054] : [0 0 1 0]
[0.38527432 0.14048474 0.22987852 0.24436246] : [0 0 1 0]
[0.30042765 0.33037364 0.17430873 0.19488999] : [1 0 0 0]
[0.4286216  0.

In [120]:
y = np.reshape(y, (prob.shape[0], -1))

total_counted = 0
correct = 0
for i in range(prob.shape[0]):
    y_index = np.argmax(y[i])
    if prob[i][y_index] == np.max(prob[i]):
        correct += 1
    total_counted += 1
print(f"percentage correct = {correct/total_counted}")

percentage correct = 0.4647168059424327
